Import Libraries

In [1]:
!pip install tensorflow-io

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.0/28.0 MB 26.0 MB/s eta 0:00:00


In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from keras.datasets import mnist
from keras import backend as k
import time
from tensorflow.keras.callbacks import EarlyStopping

# 1. Process MNIST dataset

Assign training and test data

In [3]:
batch_size = 128
num_classes = 10
img_rows, img_cols = 28,28
(x_train,y_train),(x_test,y_test) =   mnist.load_data()

11490434/11490434 [==============================] - 2s 0us/step


Reshape the images

In [4]:
if k.image_data_format()=='channels_first':
    x_train=x_train.reshape(x_train.shape[0],img_rows,img_cols,1)
    x_test=x_test.reshape(x_test.shape[0],img_rows,img_cols,1)
else:
    x_train=x_train.reshape(x_train.shape[0],img_rows,img_cols,1)
    x_test=x_test.reshape(x_test.shape[0],img_rows,img_cols,1)

input_shape=(img_rows,img_cols,1)
x_train = x_train/255.0
x_test=x_test/255.0
print('x_train shape:',x_train.shape,'\nx_test shape:',x_test.shape)

x_train shape: (60000, 28, 28, 1) 
x_test shape: (10000, 28, 28, 1)


Convert class vectors to binary class matrices

In [5]:
y_train=keras.utils.to_categorical(y_train,num_classes)
y_test=keras.utils.to_categorical(y_test,num_classes)

# 2. CNN no Attention

Design the CNN architecture

In [6]:
from keras.models import Sequential
from keras import layers

In [7]:
model=Sequential()

model.add( layers.Conv2D(32,kernel_size=(3,3),activation='relu',input_shape=input_shape) )
model.add( layers.MaxPooling2D(pool_size=(2,2)) )
model.add( layers.Dropout(0.2) )
model.add( layers.Flatten() )
model.add( layers.Dense(32,activation='relu') )
model.add( layers.Dense(num_classes,activation='softmax') )
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 13, 13, 32)        0         
                                                                 
 flatten (Flatten)           (None, 5408)              0         
                                                                 
 dense (Dense)               (None, 32)                173088    
                                                                 
 dense_1 (Dense)             (None, 10)                330       
                                                        

In [8]:
model.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.categorical_crossentropy,
              metrics=['accuracy']
             )
early_stopping = EarlyStopping(monitor='accuracy', patience=3)
tic=time.time()
model.fit(x_train,y_train,
          batch_size=batch_size,
          epochs=15,
          verbose=1,
          callbacks=[early_stopping],
          validation_data=(x_test,y_test)
          )
toc=time.time()
training_time=toc-tic

Epoch 1/15
469/469 [==============================] - 16s 11ms/step - loss: 0.2923 - accuracy: 0.9162 - val_loss: 0.1120 - val_accuracy: 0.9695
Epoch 2/15
469/469 [==============================] - 2s 5ms/step - loss: 0.0994 - accuracy: 0.9703 - val_loss: 0.0745 - val_accuracy: 0.9766
Epoch 3/15
469/469 [==============================] - 2s 5ms/step - loss: 0.0727 - accuracy: 0.9782 - val_loss: 0.0609 - val_accuracy: 0.9815
Epoch 4/15
469/469 [==============================] - 2s 4ms/step - loss: 0.0597 - accuracy: 0.9818 - val_loss: 0.0591 - val_accuracy: 0.9808
Epoch 5/15
469/469 [==============================] - 2s 4ms/step - loss: 0.0502 - accuracy: 0.9848 - val_loss: 0.0525 - val_accuracy: 0.9826
Epoch 6/15
469/469 [==============================] - 3s 6ms/step - loss: 0.0449 - accuracy: 0.9861 - val_loss: 0.0479 - val_accuracy: 0.9841
Epoch 7/15
469/469 [==============================] - 2s 5ms/step - loss: 0.0394 - accuracy: 0.9881 - val_loss: 0.0488 - val_accuracy: 0.9849
Epoc

In [9]:
tic=time.time()
test_loss, test_acc = model.evaluate(x_test,y_test)
toc=time.time()
test_time=toc-tic
print("Training Time = {} s".format(np.round(training_time, 1)))
print("Testing Time = {} ms".format(np.round(test_time*1000, 1)))
print('Test Loss = {:.2f} %:'.format(np.round(test_loss, 3)*100))
print('Test Accuracy = {:.2f} %:'.format(np.round(test_acc, 3)*100))

313/313 [==============================] - 1s 2ms/step - loss: 0.0468 - accuracy: 0.9865
Training Time = 83.2 s
Testing Time = 897.4 ms
Test Loss = 4.70 %:
Test Accuracy = 98.70 %:


# 3. CNN with Attention

Design the CNN architecture

In [10]:
inputs = layers.Input(shape=input_shape)

conv1 = layers.Conv2D(32,kernel_size=(3,3),activation='relu')(inputs)
pool1 = layers.MaxPool2D(pool_size=(2,2))(conv1)
#Attention1
attention_conv1 = layers.Conv2D(1, (3,3), padding='same', activation='sigmoid')(pool1)
attention_mul1 = layers.Multiply()([pool1, attention_conv1])
pool2 = layers.MaxPool2D(pool_size=(2,2))(attention_mul1)
##########
drop1 = layers.Dropout(0.2)(pool2)
flatten2 = layers.Flatten()(drop1)
dense2 = layers.Dense(32,activation='relu')(flatten2)
dense3 = layers.Dense(num_classes,activation='softmax')(dense2)

modelAtt = keras.Model(inputs=inputs, outputs=dense3)

modelAtt.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 28, 28, 1)]  0           []                               
                                                                                                  
 conv2d_1 (Conv2D)              (None, 26, 26, 32)   320         ['input_1[0][0]']                
                                                                                                  
 max_pooling2d_1 (MaxPooling2D)  (None, 13, 13, 32)  0           ['conv2d_1[0][0]']               
                                                                                                  
 conv2d_2 (Conv2D)              (None, 13, 13, 1)    289         ['max_pooling2d_1[0][0]']        
                                                                                              

In [11]:
modelAtt.compile(optimizer=keras.optimizers.Adam(),
              loss= keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy']
             )
tic=time.time()
modelAtt.fit(x_train,y_train,
          batch_size=batch_size,
          epochs=15,
          verbose=1,
          callbacks=[early_stopping],
          validation_data=(x_test,y_test)
          )
toc=time.time()
training_time=toc-tic

Epoch 1/15
469/469 [==============================] - 5s 6ms/step - loss: 0.5070 - accuracy: 0.8476 - val_loss: 0.1868 - val_accuracy: 0.9501
Epoch 2/15
469/469 [==============================] - 2s 5ms/step - loss: 0.1849 - accuracy: 0.9445 - val_loss: 0.1162 - val_accuracy: 0.9663
Epoch 3/15
469/469 [==============================] - 2s 5ms/step - loss: 0.1373 - accuracy: 0.9587 - val_loss: 0.0864 - val_accuracy: 0.9741
Epoch 4/15
469/469 [==============================] - 2s 5ms/step - loss: 0.1105 - accuracy: 0.9656 - val_loss: 0.0733 - val_accuracy: 0.9765
Epoch 5/15
469/469 [==============================] - 3s 6ms/step - loss: 0.0936 - accuracy: 0.9703 - val_loss: 0.0614 - val_accuracy: 0.9811
Epoch 6/15
469/469 [==============================] - 3s 5ms/step - loss: 0.0838 - accuracy: 0.9736 - val_loss: 0.0582 - val_accuracy: 0.9815
Epoch 7/15
469/469 [==============================] - 2s 5ms/step - loss: 0.0761 - accuracy: 0.9758 - val_loss: 0.0530 - val_accuracy: 0.9818
Epoch 

In [12]:
tic=time.time()
test_loss, test_acc = modelAtt.evaluate(x_test,y_test)
toc=time.time()
test_time=toc-tic
print("Training Time = {} s".format(np.round(training_time, 1)))
print("Testing Time = {} ms".format(np.round(test_time*1000, 1)))
print('Test Loss = {:.2f} %:'.format(np.round(test_loss, 3)*100))
print('Test Accuracy = {:.2f} %:'.format(np.round(test_acc, 3)*100))

313/313 [==============================] - 1s 3ms/step - loss: 0.0381 - accuracy: 0.9868
Training Time = 42.3 s
Testing Time = 924.0 ms
Test Loss = 3.80 %:
Test Accuracy = 98.70 %:
